In [72]:
import pandas as pd
import plotly.express as px

## Is the allocation equitable based on the population of each of the States and Territories, or is bias apparent?

### Show the first 5 State, Teritory or Tribal Nation and their total population in billions

In [81]:
df =pd.read_csv('https://raw.githubusercontent.com/Kingtilon1/DATA608/main/IIJA%20FUNDING%20AS%20OF%20MARCH%202023(1).csv?token=GHSAT0AAAAAACNN47NPIB7V2XF7JJND443CZNZIXUQ')
df.rename(columns={'State, Teritory or Tribal Nation': 'location'}, inplace= True)
df.head()

,location,Total (Billions)
0,ALABAMA,3.0000
1,ALASKA,3.7000
2,AMERICAN SAMOA,0.0686
3,ARIZONA,3.5000
4,ARKANSAS,2.8000


###  Looking at the distribution of the locations along with how much money in billions each location makes

In [55]:
fig = px.bar(df, x='location', y='Total (Billions)')
fig.show()

### We see just visually that alot of the locations range from 0 to 8 billion dollars with a few noticeable outliers such as California, New York and Texas, note that all three states are ranked within the top 4 of highly populated states in the USA, so far there doesn't seem to be an aparant bias because the amount in billions is very similary proportianal to the population, as in as the population decreases, so does the total amount allocated.

### Retreiving up to date population data

In [56]:

currentdf=pd.read_csv('https://raw.githubusercontent.com/Kingtilon1/DATA608/main/state_population.csv')
currentdf.rename(columns={'State': 'location'}, inplace= True)
currentdf['location']=currentdf['location'].str.upper()
currentdf.head()


,location,Population
0,ALABAMA,5108468
1,ALASKA,733406
2,AMERICAN SAMOA,43914
3,ARIZONA,7431344
4,ARKANSAS,3067732


### Merging both data frames to now carry the total amount in billions,location, population, and per capita as columns

In [91]:
combined= pd.merge(df, currentdf, on='location')
combined=combined.sort_values(by='Population')
## create a per capita column 
combined['Total (Billions)'] *= 1_000_000_000 
combined['per capita'] = combined['Total (Billions)'] / combined['Population']

In [102]:
fig = px.bar(combined, x='location', y='per capita',
             hover_data=['Population', 'Total (Billions)','per capita'], color='Total (Billions)',
             labels={'pop':'Population of US States/Territiories'}
             )
fig.show()

### Note that the graph is going in order for smallest population to largest, with the colors representing the total amount in the billions per year, and although the states with the higher population are on the right side of the graph, the highest percapita is somewhat right skewed, this doesn't prove a bias however because other  things have to be taken into consideration when determing the gdp of a state, and if there's bias such as economic activities, industry specialization, infrastructure investment, education levels, access to markets, natural resource endowment, and regional development policies

## Does the allocation favor the political interests of the Biden administration?

In [ ]:
## storing every  political affiliation with the relevent location based off of 2020 election results
political_affiliation = {
    'Alabama': 'Republican', 'Alaska': 'Republican', 'American Samoa': 'Democrat',
    'Arizona': 'Democrat', 'Arkansas': 'Republican', 'California': 'Democrat',
    'Colorado': 'Democrat', 'Connecticut': 'Democrat', 'Delaware': 'Democrat',
    'District of Columbia': 'Democrat', 'Florida': 'Republican', 'Georgia': 'Republican',
    'Guam': 'Democrat', 'Hawaii': 'Democrat', 'Idaho': 'Republican', 'Illinois': 'Democrat',
    'Indiana': 'Republican', 'Iowa': 'Republican', 'Kansas': 'Republican', 'Kentucky': 'Republican',
    'Louisiana': 'Republican', 'Maine': 'Democrat', 'Maryland': 'Democrat', 'Massachusetts': 'Democrat',
    'Michigan': 'Democrat', 'Minnesota': 'Democrat', 'Mississippi': 'Republican', 'Missouri': 'Republican',
    'Montana': 'Republican', 'Nebraska': 'Republican', 'Nevada': 'Democrat', 'New Hampshire': 'Democrat',
    'New Jersey': 'Democrat', 'New Mexico': 'Democrat', 'New York': 'Democrat', 'North Carolina': 'Republican',
    'North Dakota': 'Republican', 'Northern Mariana Islands': 'Democrat', 'Ohio': 'Republican',
    'Oklahoma': 'Republican', 'Oregon': 'Democrat', 'Pennsylvania': 'Democrat', 'Puerto Rico': 'Democrat',
    'Rhode Island': 'Democrat', 'South Carolina': 'Republican', 'South Dakota': 'Republican',
    'Tennessee': 'Republican', 'Texas': 'Republican', 'Tribal Communities': 'Democrat',
    'US Virgin Islands': 'Democrat', 'Utah': 'Republican', 'Vermont': 'Democrat', 'Virginia': 'Democrat',
    'Washington': 'Democrat', 'West Virginia': 'Republican', 'Wisconsin': 'Democrat', 'Wyoming': 'Republican'
}

adjusted_political_affiliation = {location.upper(): party for location, party in political_affiliation.items()}

# Assign political affiliations based on the adjusted dictionary
combined['Party'] = combined['location'].map(adjusted_political_affiliation)

# Displaying the DataFrame to verify the changes
print(combined)

In [104]:
fig = px.bar(combined, x='location', y='per capita',
             hover_data=['Population', 'Total (Billions)','per capita'], color='Party',
             labels={'pop':'Population of US States/Territiories'}
             )
fig.show()

### Based off of this bar chart we notice that its actually the locations that voted republican that have a higher gdp per capita, so I would not lean towards the idea of their being a bias in allocation of money towards the Biden administration seeing that its the republican states that have a higher gpd pre capita

In [106]:
party_gdp = combined.groupby('Party')['per capita'].sum()

total_per_capita_gdp = party_gdp.sum()

party_percentage = (party_gdp / total_per_capita_gdp) * 100

party_percentage = party_percentage.reset_index()

party_percentage = party_percentage.merge(combined.groupby('Party')['Population'].sum().reset_index(), on='Party')

fig = px.bar(party_percentage, x='Party', y='per capita',
             labels={'per capita': 'Percentage of Total Per Capita GDP (%)'},
             color='Party', color_discrete_map={'Democrat': 'blue', 'Republican': 'red'},
             title='Percentage of Total Per Capita GDP by Political Affiliation',
             hover_data={'Population': True, 'Party': False})
fig.show()

### In conclusion, we see that, although the Democrat locations have a higher population, the republican states territores have a higher percentage of total per capita which is %56.14 with Democrtats having 43.86